# Evolution

Compute random graphs and statistics from multiple tournesol datasets of different dates to compare them

## 1. Init or update Tournesol dataset (Todo once every week)

- Download zip from https://api.tournesol.app/exports/all/
- Extract zip
	- Recommended to do so in directories names with the week date like: `tournesol_dataset_2023-12-31/`

## 2. Init or update Youtube cache (if tournesol dataset was updated)

Run script `py src/rndstats.py (-t <latest TOURNESOL_DATASET_PATH>) (-c <YTDATA_CACHE_PATH>) --fetch`

## 3. Check and update Notebook variables

See bloc [2] below

In [ ]:
# Imports
import os
import re
import matplotlib.dates as mtdt
import html
import plotly.graph_objects as go
import colorsys
import math
import numpy as np
import matplotlib.pyplot as plt
import sys
from datetime import datetime
from matplotlib.ticker import ScalarFormatter
from numpy import std
from typing import Iterable, Iterator
from matplotlib.axis import Axis
from matplotlib.figure import Figure
from matplotlib.sankey import Sankey
from IPython.display import SVG

# Project imports
sys.path.insert(1, '../src')
from model.youtube_api import YTData
from model.users import extractAllTournesolUsers, TournesolUser
from model.comparisons import ComparisonFile, ComparisonLine
from model.collectivecriteriascores import CollectiveCriteriaScoresFile
from model.individualcriteriascores import IndividualCriteriaScoresFile


In [ ]:
# Notebook Variables
TOURNESOL_DATASETS_PARENT_PATH='../data' # Expecting data files located into: <TOURNESOL_DATASETS_PARENT_PATH>/<TOURNESOL_DATASET_PATH_FORMAT>/*.csv
TOURNESOL_DATASET_PATH_FORMAT=r'tournesol_dataset_([0-9]{4}-[0-9]{2}-[0-9]{2})' # SHOULD END in "yyyy-mm-dd"
YTDATA_CACHE_PATH='../data/YTData_cache.json'

In [ ]:
# Constants
TOURNESOL_DATASET_PATHS: dict[str, str]={} # date: path
for f in os.listdir(TOURNESOL_DATASETS_PARENT_PATH):
	match = re.match(TOURNESOL_DATASET_PATH_FORMAT, f)
	if match:
		TOURNESOL_DATASET_PATHS[match.group(1)] = os.path.abspath(TOURNESOL_DATASETS_PARENT_PATH+'/'+f)
DATE_MAP = {s: datetime.strptime(s, '%Y-%m-%d').date() for s in TOURNESOL_DATASET_PATHS}
SORTED_DATES = sorted((k,v) for (k,v) in DATE_MAP.items())
LAST = SORTED_DATES[-1][0]
print('Dataset snapshots found:', len(TOURNESOL_DATASET_PATHS), f"(from {min(TOURNESOL_DATASET_PATHS.keys())} to {max(TOURNESOL_DATASET_PATHS.keys())})")

# Data files
USERS:set[TournesolUser] = extractAllTournesolUsers(TOURNESOL_DATASET_PATHS[LAST])
COMPARISONS:dict[str,ComparisonFile] = dict()
COLLECTIVE_SCORES:dict[str,CollectiveCriteriaScoresFile] = dict()
INDIVIDUAL_SCORES:dict[str,IndividualCriteriaScoresFile] = dict()
for date in TOURNESOL_DATASET_PATHS:
	COMPARISONS[date] = ComparisonFile(TOURNESOL_DATASET_PATHS[date])
	COLLECTIVE_SCORES[date] = CollectiveCriteriaScoresFile(TOURNESOL_DATASET_PATHS[date])
	INDIVIDUAL_SCORES[date] = IndividualCriteriaScoresFile(TOURNESOL_DATASET_PATHS[date])

# Youtube Data
YTDATA = YTData()
try:
	YTDATA.load(YTDATA_CACHE_PATH)
except FileNotFoundError as e:
	pass

In [ ]:
## User against global scores

USER='NatNgs'
CRITERION='largely_recommended'
MIN_DATE = '2023-07-01'

vids:dict[str,dict[str,tuple[float,float,int]]] = dict() # vids[vid][date] = (glob_score, personnal_score, nb_users)

for d in INDIVIDUAL_SCORES:
	indiv = INDIVIDUAL_SCORES[d].get_scores(criterion=CRITERION, users=[USER])[USER] # `(score, uncertainty) = out[video][criterion]`
	glob = COLLECTIVE_SCORES[d].get_scores(criterion=CRITERION, vids=indiv.keys()) # `(score, uncertainty) = out[video][criterion]`
	for vid in indiv:
		vids.setdefault(vid, dict()).setdefault(d, (indiv[vid][CRITERION][0], glob[vid][CRITERION][0], None))

for d in INDIVIDUAL_SCORES:
	scores = INDIVIDUAL_SCORES[d].get_scores(criterion=CRITERION, vids=vids.keys()) # `(score, uncertainty) = out[user][video][criterion]`
	for vid in vids:
		if d in vids[vid]:
			vids[vid][d] = (vids[vid][d][0], vids[vid][d][1], len([u for u in scores if vid in scores[u]]))

# print(vids)

anim_dates = [d[0] for d in SORTED_DATES if d[0] >= MIN_DATE]

step_duration = 2.0 # seconds
steps_count = len(anim_dates)-1
anim_duration = f"{steps_count*step_duration:0.0f}s"
keyTimes = [i/steps_count for i in range(steps_count)]

def make_svg_point(vid:str):
	tt = list(keyTimes)
	xx = []
	yy = []
	rr = []
	for d in anim_dates:
		(y,x,r) = vids[vid].get(d,(0,0,0))
		xx.append(x)
		yy.append(-y)
		rr.append(r)
	
	rr = [math.log10(r+1) for r in rr]

	fill=f"{hash(vid) % (1<<24):6X}"
	animx = \
		  f"dur=\"{anim_duration}\" " \
		+ f"values=\"{';'.join(f'{x:0.2f}'.rstrip('0').rstrip('.') for x in xx)}\" " \
		+ f"keyTimes=\"{';'.join(f'{t:0.2f}'.rstrip('0').rstrip('.') for t in tt)}\""
	animy = \
		  f"dur=\"{anim_duration}\" " \
		+ f"values=\"{';'.join(f'{y:0.2f}'.rstrip('0').rstrip('.') for y in yy)}\" " \
		+ f"keyTimes=\"{';'.join(f'{t:0.2f}'.rstrip('0').rstrip('.') for t in tt)}\""
	animr = \
		  f"dur=\"{anim_duration}\" " \
		+ f"values=\"{';'.join(f'{r:0.2f}'.rstrip('0').rstrip('.') for r in rr)}\" " \
		+ f"keyTimes=\"{';'.join(f'{t:0.2f}'.rstrip('0').rstrip('.') for t in tt)}\""
	title = vid # f"{YTDATA.videos.get(vid, vid)}"

	return (
		f"<circle cx=\"{xx[-1]}\" cy=\"{yy[-1]}\" r=\"{rr[-1]}\" fill=\"#{fill}\"><animate attributeName=\"cx\" {animx}/><animate attributeName=\"cy\" {animy}/><animate attributeName=\"r\" {animr}/></circle>",
		f"<text x=\"{xx[-1]}\" y=\"{yy[-1]}\">{html.escape(title)}<animate attributeName=\"x\" {animx}/><animate attributeName=\"y\" {animy}/></text>"
	)

# Begin
svg = '<svg viewBox="-100 -100 200 200" xmlns="http://www.w3.org/2000/svg">'
svg += '<g stroke="black" stroke-width="0.1"><line x1="-100" y1="0" x2="100" y2="0"/><line x1="0" y1="-100" x2="0" y2="100"/></g>'

# Legend
svg += f'<g font-size="1"><text x="-99" y="1">Collective -100</text><text x="99" y="-1" text-anchor="end">+100 Collective</text><text x="1" y="-99">+100 {USER}</text><text x="-1" y="99" text-anchor="end">{USER} -100</text></g>'

# Elements
svg_circles = []
svg_texts = []
for vid in vids:
	(c,t) = make_svg_point(vid)
	svg_circles.append(c)
	svg_texts.append(t)
svg += '<g font-size="1" repeatCount="1">' + ''.join(svg_texts) + '</g>'
svg += '<g repeatCount="1" opacity=".75">' + ''.join(svg_circles) + '</g>'

# End
svg += '</svg>'

print(svg)

display(SVG(data=svg))

In [ ]:
## Top & growing

prev_scores = None
for d,_ in SORTED_DATES:
	scores = COLLECTIVE_SCORES[d].get_scores(criterion='largely_recommended') # (score, uncertainty) = allscores[video]['largely_recommended']
	if not prev_scores:
		prev_scores = scores
		continue

	for vid in list(scores.keys()):
		if not vid in prev_scores:
			prev_scores[vid] = scores[vid]
	
	growth:list[tuple[float,str]] = list(map(lambda vid: (abs(scores[vid]['largely_recommended'][0] - prev_scores[vid]['largely_recommended'][0]), vid), scores.keys())) ## + & -
	#growth:list[tuple[float,str]] = list(map(lambda vid: (scores[vid]['largely_recommended'][0] - prev_scores[vid]['largely_recommended'][0], vid), scores.keys())) ## Growing only
	#growth:list[tuple[float,str]] = list(map(lambda vid: (prev_scores[vid]['largely_recommended'][0] - scores[vid]['largely_recommended'][0], vid), scores.keys())) ## Falling only
	growth.sort(reverse=True)

	print('Week ' + d)
	for i in range(10):
		vid = growth[i][1]
		prev = prev_scores[vid]['largely_recommended'][0]
		news = scores[vid]['largely_recommended'][0]
		if news > prev:
			print(f"{i+1:2d}: ({prev:+3.0f} => {news:+3.0f} ↗{growth[i][0]:4.1f})", YTDATA.videos[vid])
		else:
			print(f"{i+1:2d}: ({prev:+3.0f} => {news:+3.0f} ↘{growth[i][0]:4.1f})", YTDATA.videos[vid])
	print()
	prev_scores = scores


In [ ]:
## Video score over time (part 1)
# Plot lines: one line per video (opacity: number of users having compared the video - 1=0%..10+=100%)
# x axis: Number of others videos it has been compared with
# y axis: Score (-100 to +100)

# Fetch scores
vid_date_vals:dict[str,dict[str,tuple[int,int,float]]] = dict() # vid_date_vals[vid][date] = (users,vid_cmps,score)
max_cmps = 0
max_usrs = 0

def fetch_collective_scores():
	vid_date_data:dict[str,dict[str,tuple[set[str],set[str]]]] = dict() # vid_date_data[vid][date] = (users,vid_cmps)
	# Fetch users & comparisons per date
	def fetch_user_data(line: ComparisonLine):
		if line.criterion != 'largely_recommended':
			return
		vid1_d = vid_date_data.setdefault(line.vid1, dict()).setdefault(line.date, (set(),set()))
		vid1_d[0].add(line.user)
		vid1_d[1].add(line.vid2)
		vid2_d = vid_date_data.setdefault(line.vid2, dict()).setdefault(line.date, (set(),set()))
		vid2_d[0].add(line.user)
		vid2_d[1].add(line.vid1)
	print('Fetching Comparisons', end='')
	for comparisons in COMPARISONS.values():
		print('.', end='', flush=True)
		comparisons.foreach(fetch_user_data)
	print()

	print('Fetching scores', end='')
	for (date,collectivescore) in COLLECTIVE_SCORES.items():
		print('.', end='', flush=True)
		scores = collectivescore.get_scores(criterion='largely_recommended', vids=vid_date_data.keys()) # (score,uncertainty) = out[vid]['largely_recommended']
		for vid in scores:
			usrs = set()
			cmps = 0
			for d in vid_date_data[vid]:
				if d <= date:
					usrs.update(vid_date_data[vid][d][0])
					cmps += len(vid_date_data[vid][d][1])
			s = scores[vid]['largely_recommended'][0]

			# Ignore if less than 3 users
			if len(usrs) < 3: continue
			if len(usrs) > max_usrs: max_usrs = len(usrs)
			if cmps > max_cmps: max_cmps = cmps

			vid_date_vals.setdefault(vid,dict())[date] = (len(usrs),cmps,s)
	print()
	return (max_usrs, max_cmps)

def fetch_individual_scores():
	USER = 'NatNgs'

	vid_date_data:dict[str,dict[str,set[str]]] = dict() # vid_date_data[vid][date] = vid_cmps
	# Fetch users & comparisons per date
	def fetch_user_data(line: ComparisonLine):
		if line.criterion != 'largely_recommended' or line.user != USER:
			return
		vid_date_data.setdefault(line.vid1, dict()).setdefault(line.date, set()).add(line.vid2)
		vid_date_data.setdefault(line.vid2, dict()).setdefault(line.date, set()).add(line.vid1)
	print(f'Fetching {USER} Comparisons', end='')
	for comparisons in COMPARISONS.values():
		print('.', end='', flush=True)
		comparisons.foreach(fetch_user_data)
	print()

	max_cmps = 0
	print(f'Fetching {USER} scores', end='')
	for (date,individualscores) in INDIVIDUAL_SCORES.items():
		print('.', end='', flush=True)
		scores = individualscores.get_scores(criterion='largely_recommended', vids=vid_date_data.keys(), users=[USER])[USER] # (score,uncertainty) = out[vid]['largely_recommended']
		for vid in scores:
			cmps = 0
			for d in vid_date_data[vid]:
				if d <= date:
					cmps += len(vid_date_data[vid][d])
			s = scores[vid]['largely_recommended'][0]

			if cmps > max_cmps: max_cmps = cmps

			vid_date_vals.setdefault(vid,dict())[date] = (1,cmps,s)
	print()
	return (1,max_cmps)

(max_usrs,max_cmps) = fetch_individual_scores() # fetch_collective_scores()

In [ ]:
## Video score over time (part 2)
print('Plotting...', end='\r', flush=True)

# Prepare plot
fig, ax = plt.subplots()
fig.set_size_inches(8, 6)
# ax.set_ylabel('Collective largely_recommended score')
ax.set_ylabel('lpfaucon largely_recommended score')
ax.set_xlabel('Date')

ax.grid(visible=True, which='major', axis='both', color='gray')
ax.grid(visible=True, which='minor', axis='both', color='#eee')

ax.set_ylim([-100, 100])
ax.yaxis.set_ticks([-75,-50,-25,0,25,50,75])

ax.set_xlim(xmin=DATE_MAP['2023-08-21'], xmax=SORTED_DATES[-1][1])
ax.xaxis.set_ticks([SORTED_DATES[i][1] for i in range(len(SORTED_DATES)-1, 0, -4) if SORTED_DATES[i][0] >= '2023-08-21'])
ax.xaxis.set_ticks([s[1] for s in SORTED_DATES if s[0] >= '2023-08-21'], minor=True)
ax.xaxis.set_major_formatter(mtdt.DateFormatter('%y-%m-%d'))
#ax.set_xlim([0, max_usrs+1])
# ax.set_xscale('log')
# ax.set_xlim([1, 10**math.ceil(math.log(max_usrs,10))])
# ax.xaxis.set_major_formatter(ScalarFormatter())

# Plot lines
n=0
for (n,vid) in enumerate(vid_date_vals):
	# vid_date_vals[vid][date] = (users,vid_cmps,score)
	xx = []
	yy = []
	oo = 0
	for d in sorted(vid_date_vals[vid].keys()):
		data = vid_date_vals[vid][d] #(users,vid_cmps,score)
		if d < '2023-08-21': continue
		xx.append(DATE_MAP[d])
		yy.append(data[2])
		oo = data[0]
	oo = math.sqrt(oo) / math.sqrt(max_usrs)

	#if max(yy)-min(yy) >= 30:
	print(f"Plotting {n+1}/{len(vid_date_vals)}...   ", end='\r', flush=True)
	
	rgbcolor = colorsys.hsv_to_rgb(0 if yy[-1] < yy[0] else 0.3, 0.8, min(1,(max(yy)-min(yy))/25))
	hexcolor = "".join("%02X" % round(i*255) for i in rgbcolor)
	ax.plot_date(xx, yy, linewidth=1, fmt='-', color="#" + hexcolor + hexcolor[-2:]) # color='#0261'
		#if xx[-1] > 18:
		#ax.text(xx[-1], yy[-1], s=YTDATA.videos.get(vid,vid), horizontalalignment='left', verticalalignment='center', fontsize=6)
print()

In [ ]:
dateranks:list[list[tuple[float,str]]] = [[],[]] # dateranks[0/1][#rank] = (score,vid)

date0 = '2023-09-25'
date1 = '2023-10-02'

vids:dict[str,set[str]] = dict() # vids[vid] = users
def extractVids(line:ComparisonLine):
	if line.criterion == 'largely_recommended':
		vids.setdefault(line.vid1,set()).add(line.user)
		vids.setdefault(line.vid2,set()).add(line.user)
COMPARISONS[date0].foreach(extractVids)
for vid in list(vids.keys()):
	if len(vids[vid]) < 5:
		vids.pop(vid)

before = COLLECTIVE_SCORES[date0].get_scores(criterion='largely_recommended', vids=vids.keys()) # [vid]['largely_recommended'] = (score,uncertainty)
after = COLLECTIVE_SCORES[date1].get_scores(criterion='largely_recommended', vids=vids.keys())

for vid in vids:
	if vid in before and vid in after:
		dateranks[0].append((before[vid]['largely_recommended'][0], vid))
		dateranks[1].append((after[vid]['largely_recommended'][0], vid))

dateranks[0].sort(reverse=True)
dateranks[1].sort(reverse=True)

ranks:dict[str,tuple[int,int,int]] = dict()
for i,dt0 in enumerate(dateranks[0]):
	ranks[dt0[1]] = (i, None, None)

ordered_ranks = []
for i,dt1 in enumerate(dateranks[1]):
	ranks[dt1[1]] = (ranks[dt1[1]][0], i, i-ranks[dt1[1]][0])
	ordered_ranks.append((abs(ranks[dt1[1]][2]), ranks[dt1[1]][1], dt1[1]))
	#ordered_ranks.append((ranks[dt1[1]][1], dt1[1]))

print(len(ordered_ranks))
ordered_ranks.sort(reverse=True)
print('yt video Id: was (score, uncty), now (score, uncty), rank: before => after : change')
for dt in ordered_ranks:
	vid = dt[-1]
	print(f"{vid}: was {before[vid]['largely_recommended']}, now {after[vid]['largely_recommended']}, rank: {ranks[vid][0]+1} => {ranks[vid][1]+1} : {-ranks[vid][2]:+d}") # {YTDATA.videos.get(dt[1],dt[1])}")




-----

# Users impact

In [ ]:
# Users ranked by how much their scores changed between last 2 weeks

week1_scores = INDIVIDUAL_SCORES[SORTED_DATES[-2][0]].get_scores(criterion='largely_recommended') # [user][vid][crit] = (score, unc.)
vids = {vid for v in week1_scores.values() for vid in v}
week2_scores = INDIVIDUAL_SCORES[SORTED_DATES[-1][0]].get_scores(users=week1_scores.keys(), vids=vids) # [user][vid][crit] = (score, unc.)

users_weekscores: dict[str,list[float]] = dict() # users_weekscores[user] = [upd. score]
for usr in week1_scores:
	if len(week1_scores[usr]) < 10:
		continue
	user_weekscores = list()
	for vid in week1_scores[usr]:
		for crit in week1_scores[usr][vid]:
			if usr in week2_scores and vid in week2_scores[usr] and crit in week2_scores[usr][vid]:
				user_weekscores.append(abs(week2_scores[usr][vid][crit][0]-week1_scores[usr][vid][crit][0]))
	users_weekscores[usr] = user_weekscores

sorted_users = sorted(users_weekscores.keys(), key=lambda usr: sum(users_weekscores[usr]), reverse=True)

print('  user            sum     avg  median    max')
print('---------------  ----  ------  ------  -----')
for user in sorted_users[:10]:
	print(f"{user[:15]:<15}  {sum(users_weekscores[user]):4.0f}  {np.average(users_weekscores[user]):6.2f}  {np.quantile(users_weekscores[user],0.5):6.2f}  {max(users_weekscores[user]):5.1f}")


In [ ]:
# Users ranked by how much the score of their comparisons changed between last 2 weeks

def users_impact(datemin:str, datemax:str):
	MIN_TRUST_SCORE = 0.01 # Add this value to all trust scores to account for users with 0 trust score (to be improved by retroengineering trust algorithm, see https://arxiv.org/pdf/2211.01179.pdf)

	# Users to consider
	users_last_week_comparisons: dict[str,set[str]] = dict()# [user] = {vid, ...}
	def extractVids(line:ComparisonLine):
		if line.criterion == 'largely_recommended' and line.date >= datemin and line.date < datemax:
			user_set = users_last_week_comparisons.setdefault(line.user,set())
			user_set.add(line.vid1)
			user_set.add(line.vid2)
	COMPARISONS[LAST].foreach(extractVids)
	users = set(users_last_week_comparisons.keys())

	# Users trust score
	trust_scores = {u.public_username: u.trust_score + MIN_TRUST_SCORE for u in USERS if u.public_username in users}

	# Individual scores differences
	week1_scores = INDIVIDUAL_SCORES[datemin].get_scores(users=users, criterion='largely_recommended') # [user][vid][crit] = (score, unc.)
	week2_scores = INDIVIDUAL_SCORES[datemax].get_scores(users=users, criterion='largely_recommended') # [user][vid][crit] = (score, unc.)
	weighted_indiv_diff = dict()
	for u in users:
		if not (u in week1_scores and u in week2_scores and u in trust_scores):
			continue
		weighted_diff = {
			v: (week2_scores[u][v]['largely_recommended'][0]-week1_scores[u][v]['largely_recommended'][0])*trust_scores[u]
			for v in set(week1_scores[u].keys()).intersection(week2_scores[u].keys()) 
		}
		weighted_indiv_diff[u] = weighted_diff

	# Collective scores differences
	week1_scores = COLLECTIVE_SCORES[datemin].get_scores(criterion='largely_recommended') # [vid][crit] = (score, unc.)
	week2_scores = COLLECTIVE_SCORES[datemax].get_scores(criterion='largely_recommended') # [vid][crit] = (score, unc.)
	coll_diff = {v: week2_scores[v]['largely_recommended'][0]-week1_scores[v]['largely_recommended'][0] for v in week2_scores and week1_scores}

	# Decompose collective scores differences from individual scores
	impact_from_users:dict[str,dict[str,float]] = dict() # [user][vid] = collectiveimpact
	impact_on_videos:dict[str,dict[str,float]] = dict() # [vid][user] = collectiveimpact (such as sum of [vid].values() == coll_diff[vid])
	for v in coll_diff:
		if coll_diff[v] < 0.1: # Ignore low changes
			continue
		total_impact = 0
		users_impact = {} # [user] = individualimpact (value between -220 and +220, most of the time will be around 0)
		for u in weighted_indiv_diff:
			if not v in weighted_indiv_diff[u]:
				continue
			users_impact[u] = weighted_indiv_diff[u][v]
			total_impact += weighted_indiv_diff[u][v]
		
		# Controls to prevent errors
		if abs(total_impact) < 0.01: # No (or almost no) impact found, ignore
			print(f"Failed to compute impact for {v}: Collective score change is {coll_diff[v]:.2f} but cumulated individual change is negligeable ({total_impact})")
			continue
		if np.sign(total_impact) != np.sign(coll_diff[v]):
			print(f"Failed to compute impact for {v}: Collective score change is {coll_diff[v]:.2f} but cumulated individual change is {total_impact:.2f}")
			continue

		# Compute every user impact (dividing total impact by collective score change)
		impact_on_videos[v] = dict()
		for u in users_impact:
			coll_impact = users_impact[u]/total_impact * coll_diff[v]
			impact_on_videos[v][u] = coll_impact
			impact_from_users.setdefault(u, dict())[v] = coll_impact
	


	# Sum users absolute impact, and print top 10
	def print_user_impact(u, maximpacted=5):
		print(f"{u}: Estimated impact: +{sum(s for s in impact_from_users[u].values() if s > 0):.1f}, -{-sum(s for s in impact_from_users[u].values() if s < 0):.1f}")
		sorted_impact = [v for v in sorted(impact_from_users[u], key=lambda v: abs(impact_from_users[u][v]), reverse=True) if abs(impact_from_users[u][v] > 0.05)]
		print('    Impacted',len(sorted_impact),'videos:', ', '.join(f"{sorted_impact[i]} ({impact_from_users[u][sorted_impact[i]]:+.1f})" for i in range(0, min(maximpacted, len(sorted_impact)))), '...' if len(sorted_impact) > maximpacted else '')
	
	def print_video_impact(v, maximpacted=5):
		print(f"{v}: Score {week1_scores[v]['largely_recommended'][0]:.1f} => {week2_scores[v]['largely_recommended'][0]:.1f} (+{sum(s for s in impact_on_videos[v].values() if s > 0):.1f}, -{-sum(s for s in impact_on_videos[v].values() if s < 0):.1f})")
		sorted_impact = [u for u in sorted(impact_on_videos[v], key=lambda u: abs(impact_on_videos[v][u]), reverse=True) if abs(impact_on_videos[v][u]) > 0.05]
		print('    Impacted by',len(sorted_impact),'user(s):', ', '.join(f"{sorted_impact[i]} ({impact_on_videos[v][sorted_impact[i]]:+.1f})" for i in range(0, min(maximpacted, len(sorted_impact)))), '...' if len(sorted_impact) > maximpacted else '')

	print()
	print('User impacts:')
	cumulated_user_impact = {u: np.sum(np.abs(list(impact_from_users[u].values()))) for u in impact_from_users}
	for u in sorted(cumulated_user_impact, key=cumulated_user_impact.get, reverse=True)[:10]:
		print_user_impact(u)
	print()
	print('Most impacted videos:')
	cumulated_video_impact = {v: np.sum(np.abs(list(impact_on_videos[v].values()))) for v in impact_on_videos}
	for v in sorted(cumulated_video_impact, key=cumulated_video_impact.get, reverse=True)[:10]:
		print_video_impact(v)

	print()
	print_user_impact('NatNgs', 999)

users_impact(SORTED_DATES[-2][0], SORTED_DATES[-1][0])



-----

# Returning users

In [ ]:
"""
ids = ['new', 'w1', 'w2']
steps = [
	# Source / Target / Value / Label / color
	('new', 'w1', .1, '>100', 'rgba(200,0,0,0.25)'),
	('w1', 'w2', .6, '>100', 'rgba(200,0,0,0.25)'),
	('new', 'w2', .1, '>100', 'rgba(200,0,0,0.25)'),
	('new', 'w1', .2, '10-100', 'rgba(150,150,0,0.25)'),
	('w1', 'w2', .8, '10-100', 'rgba(150,150,0,0.25)'),
	('new', 'w2', .2, '10-100', 'rgba(150,150,0,0.25)'),
	('new', 'w1', .1, '<10', 'rgba(100,100,100,0.25)'),
	('w1', 'w2', .2, '<10', 'rgba(100,100,100,0.25)'),
	('new', 'w2', .1, '<10', 'rgba(100,100,100,0.25)')
]
"""
ids = ['<2023-09', 'new']

cmps:dict[str,dict[str,int]] = dict() # [user][date] = nbCmps
def extractCmps(line: ComparisonLine):
	cmps.setdefault(line.user, dict()).setdefault(line.date, 0)
	cmps[line.user][line.date] += 1
COMPARISONS[LAST].foreach(extractCmps)

categories = [('<=10', 'rgba(100,100,100,0.25)'),('10-100', 'rgba(150,150,0,0.25)'), ('>=100', 'rgba(200,0,0,0.25)')]
extracted_data: dict[str,dict[str,list[int]]] = dict() # data[Source][Target] = [<10, 10-100, >100]
for u in cmps:
	prev = 'new'
	for d in sorted(cmps[u]):
		if not d in ids:
			ids.append(d)
		nb = cmps[u][d]
		
		cat = 0
		if nb <= 10:
			pass
		elif nb < 100:
			cat = 1
		else:
			cat = 2

		extracted_data.setdefault(prev, dict()).setdefault(d, [0,0,0])[cat] += 1
		prev = d

steps = [] # (Source, Target, Value, Label, color)
for source in extracted_data:
	for dest in extracted_data[source]:
		if dest < '2023-09':
			continue
		vsource = '<2023-09' if source < '2023-09' else source
		for (cat,count) in enumerate(extracted_data[source][dest]):
			steps.append((vsource, dest, count, categories[cat][0], 'rgba(127,127,127,0.01)' if vsource == '<2023-09' else categories[cat][1]))

steps.sort(key=lambda s:(s[4], s[0], s[1]))

fig = go.Figure(go.Sankey(
	node = dict(
		label=ids,
		pad=10
		# groups=[[0],[1],list(range(2,len(ids)))]
	),
	link = dict(
		source = [ids.index(s[0]) for s in steps],
		target = [ids.index(s[1]) for s in steps],
		value = [s[2] for s in steps],
		label = [s[3] for s in steps],
		color = [s[4] for s in steps],
		# arrowlen=10
	)
))
fig.update_layout(
	autosize=False,
	width=1000,
	height=750
)

In [ ]:
"""
ids = ['new', 'w1', 'w2']
steps = [
	# Source / Target / Value / Label / color
	('new', 'w1', .1, '>100', 'rgba(200,0,0,0.25)'),
	('w1', 'w2', .6, '>100', 'rgba(200,0,0,0.25)'),
	('new', 'w2', .1, '>100', 'rgba(200,0,0,0.25)'),
	('new', 'w1', .2, '10-100', 'rgba(150,150,0,0.25)'),
	('w1', 'w2', .8, '10-100', 'rgba(150,150,0,0.25)'),
	('new', 'w2', .2, '10-100', 'rgba(150,150,0,0.25)'),
	('new', 'w1', .1, '<10', 'rgba(100,100,100,0.25)'),
	('w1', 'w2', .2, '<10', 'rgba(100,100,100,0.25)'),
	('new', 'w2', .1, '<10', 'rgba(100,100,100,0.25)')
]
"""
OLD='2023-07'
ids = ['<'+OLD]

cmps:dict[str,dict[str,int]] = dict() # [user][date] = nbCmps
def extractCmps(line: ComparisonLine):
	cmps.setdefault(line.user, dict()).setdefault(line.date, 0)
	cmps[line.user][line.date] += 1
COMPARISONS[LAST].foreach(extractCmps)

categories = [('<=10', 'rgba(100,100,100,0.25)'),('10-100', 'rgba(150,150,0,0.25)'), ('>=100', 'rgba(200,0,0,0.25)')]
extracted_data: dict[str,dict[str,list[int]]] = dict() # data[Source][Target] = [<10, 10-100, >100]
for u in cmps:
	prev = None
	for d in sorted(cmps[u]):
		if not d in ids:
			ids.append(d)
		if prev is None:
			prev = 'new-' + d
			if not prev in ids:
				ids.append(prev)
		elif prev < OLD:
			prev = '<' + OLD
		
		if d > OLD:
			nb = cmps[u][d]
			
			cat = 0
			if nb <= 10:
				pass
			elif nb < 100:
				cat = 1
			else:
				cat = 2

			extracted_data.setdefault(prev, dict()).setdefault(d, [0,0,0])[cat] += 1
		prev = d

steps = [] # (Source, Target, Value, Label, color)
for source in extracted_data:
	for dest in extracted_data[source]:
		for (cat,count) in enumerate(extracted_data[source][dest]):
			steps.append((source, dest, count, categories[cat][0], 'rgba(127,127,127,0)' if source == '<' + OLD else categories[cat][1]))

steps.sort(key=lambda s:(s[4], s[0], s[1]))

fig = go.Figure(go.Sankey(
	node = dict(
		label=ids,
		pad=10
		# groups=[[0],[1],list(range(2,len(ids)))]
	),
	link = dict(
		source = [ids.index(s[0]) for s in steps],
		target = [ids.index(s[1]) for s in steps],
		value = [s[2] for s in steps],
		label = [s[3] for s in steps],
		color = [s[4] for s in steps],
		# arrowlen=10
	)
))
fig.update_layout(
	autosize=False,
	width=1000,
	height=750
)

In [ ]:
# Table: Returning users

cmps:dict[str,dict[str,int]] = dict() # [user][date] = nbCmps
def extractCmps(line: ComparisonLine):
	if line.criterion == 'largely_recommended':
		cmps.setdefault(line.user, dict()).setdefault(line.date, 0)
		cmps[line.user][line.date] += 1
COMPARISONS[LAST].foreach(extractCmps)

categories = {
	'total':   {SORTED_DATES[-i][0] for i in range(0  , len(SORTED_DATES)) },
	'new':     {SORTED_DATES[-i][0] for i in range(1  , 3                ) },
	'active':  {SORTED_DATES[-i][0] for i in range(3  , 4                ) },
	'ret_1w':  {SORTED_DATES[-i][0] for i in range(4  , 5                ) },
	'ret_1m':  {SORTED_DATES[-i][0] for i in range(5  , 9                ) },
	'ret_old': {SORTED_DATES[-i][0] for i in range(9  , len(SORTED_DATES)) },
}

tabledata: dict[str,dict[str,int]] = {k:{'usrs':0, 'cmps':0} for k in categories} # [category_key]['usrs'/'cmps'] = int
for u in cmps:
	lastdate = sorted(cmps[u])[-1]
	prevdate = sorted(cmps[u])[-2] if len(cmps[u]) > 1 else SORTED_DATES[-1][0]
	if lastdate < SORTED_DATES[-2][0]:
		continue

	for (catk, catv) in categories.items():
		if prevdate in catv:
			tabledata[catk]['usrs'] += 1
			tabledata[catk]['cmps'] += cmps[u][lastdate]

# Print
"""
      | 2023-12-31 | 2023-... | ...
TOTAL |         42 |       69 | ...
NEW   |         33 |       26 | ...
LAST  |            |          | ...
2W    |            |          | ...
3W    |            |          | ...
1M    |            |          | ...
2M    |            |          | ...
1Y    |            |          | ...
OLDER |            |          | ...
"""

print(f"| {SORTED_DATES[-2][0]} |  Users | Comparisons |")
for k,v in tabledata.items():
	print(f"| {k:10s} | {v['usrs']:6d} | {v['cmps']:11d} |")

